# Lab 3
+ ## Автор: Роман Кривохижа
+ ## Група: ІС-72
+ ## Викладач: Новікова П.А.

****
****
****

## Module importing

In [122]:
import numpy as np
import pandas as pd

In [123]:
import matplotlib.pyplot as plt
import seaborn as sns

In [124]:
sns.set_style('darkgrid')
%matplotlib inline

## Algorithm implementation

+ Алгоритм, який відтворює функціонування системи, за допомогою комп’ютерної програми називається **алгоритмом імітації**.

+ Вимоги на обслуговування генеруються елементом **CREATE** і відправляються на обслуговування до елементу **PROCESS**, який здійснює обслуговування з часовою затримкою, заданою випадковим числом, та обмеження на довжину черги, заданим невід’ємним числом

<font size="4">Ймовірність відмови: $P = \frac{N_{unserv}}{N_{all}}$</font>

<font size="4">Середня довжина черги: $L_{aver} = \frac{\sum_{i} L_i \Delta t_i}{T_{mod}}$</font>

<font size="4">Середній час очікування: $Q_{aver} = \frac{\sum_{i} L_i \Delta t_i}{N_{serv}}$</font>

<img src="diag.jpg">

In [125]:
class Rand:
    """
    Генерація випадкового числа за заданим законом розподілу
    """
    @staticmethod
    def exp(time_mean):
        a = 0
        while a == 0:
            a = np.random.rand()
        return -time_mean * np.log(a)
    
    @staticmethod
    def unif(time_min, time_max):
        a = 0
        while a == 0:
            a = np.random.rand()
        a = time_min + a * (time_max - time_min)
        return a
    
    @staticmethod
    def norm(time_mean, time_deviation):
        return np.random.normal(loc=time_mean, scale=time_deviation)

In [126]:
class Element:
    id_curr = 0
    
    def __init__(self, name=None, delay_mean=1., delay_dev=0., distribution=''):
        self.tnext = 0.0
        self.delay_mean = delay_mean
        self.delay_dev = delay_dev
        self.distribution = distribution
        self.quantity = 0
        self.tcurr = self.tnext
        self.state = 0
        self.next_elements = None
        self.id_curr = Element.id_curr
        Element.id_curr += 1
        self.name = f'element_{self.id_curr}' if name is None else name
        
    def get_delay(self):
        if self.distribution == 'exp':
            return Rand.exp(self.delay_mean)
        elif self.distribution == 'unif':
            return Rand.unif(self.delay_mean, self.delay_dev)
        elif self.distribution == 'norm':
            return Rand.norm(self.delay_mean, self.delay_dev)
        else:
            return self.delay_mean
        
    def in_act(self):
        pass
    
    def out_act(self):
        self.quantity += 1
        
    def print_info(self):
        print(f'{self.name}: state={self.state}; quantity={self.quantity}; tnext={self.tnext}')
        
    def print_result(self):
        print(f'{self.name}: state={self.state}; quantity={self.quantity};\n')
    
    def do_statistics(self, delta):
        pass

In [127]:
class Create(Element):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def out_act(self):
        # виконуємо збільшення лічильника кількості
        super().out_act()
        # встановлюємо коли пристрій буде вільним
        self.tnext = self.tcurr + super().get_delay()
        # передаємо створену заявку на настпуні елемент
        for next_element in self.next_elements:
            next_element.in_act()

In [128]:
class Process(Element):
    def __init__(self, maxqueue=np.inf, **kwargs):
        super().__init__(**kwargs)
        self.queue = 0
        self.maxqueue = maxqueue
        self.mean_queue_length = self.queue
        self.max_obs_queue_length = self.queue
        self.failure = 0
        self.tnext = np.inf
        
    def in_act(self):
        # перевіряємо чи вільний пристрій
        if self.state == 0:
            # позначаємо що пристрій зайнятий
            self.state = 1
            # встановлюємо коли пристрій буде вільним
            self.tnext = self.tcurr + super().get_delay()
        elif self.queue < self.maxqueue:
            self.queue += 1
            if self.queue > self.max_obs_queue_length:
                self.max_obs_queue_length = self.queue
        else:
            self.failure += 1
            
    def out_act(self):
        # виконуємо збільшення лічильника кількості
        super().out_act()
        # позначаємо, що елемент вільний
        self.tnext = np.inf
        self.state = 0
        
        # дістаємо з черги елемент, якщо він там є
        if self.queue > 0:
            self.queue -= 1
            self.state = 1
            self.tnext = self.tcurr + super().get_delay()
        elif self.next_elements is not None:
            for next_element in self.next_elements:
                next_element.in_act()
        
    def print_info(self):
        super().print_info()
        print(f'failure={self.failure}')
        
    def do_statistics(self, delta):
        self.mean_queue_length =+ delta * self.queue

In [129]:
class Model():
    def __init__(self, elements=[], print_logs=False):
        self.elements = elements
        self.tnext = 0
        self.tcurr = self.tnext
        self.event = elements[0]
        self.print_logs = print_logs
        
    def simulate(self, time):
        self.max_time = time
        while self.tcurr < self.max_time:
            self.tnext = np.inf
            for e in self.elements:
                # знайти найменший з моментів часу
                if e.tnext < self.tnext:
                    self.tnext = e.tnext
                    self.event = e
            if self.print_logs:
                print(f'\nIt`s time for event in {self.event.name}; time={self.tnext}')
            
            # обраховуємо статистики
            for e in self.elements:
                e.do_statistics(self.tnext - self.tcurr)
                
            # робимо переміщення до моменту завершення
            self.tcurr = self.tnext
            for e in self.elements:
                e.tcurr = self.tcurr
            
            # виконуємо операцію завершення
            self.event.out_act()
            
            # Щоб зменшити обсяг обчислень, введемо також здійснення відповідної події для всіх елементів,
            # час наступної події яких співпадає з поточним моментом часу
            for e in self.elements:
                if e.tnext == self.tcurr:
                    e.out_act()
            if self.print_logs:        
                self.print_info()
        return self.print_result()
        
    def print_info(self):
        for e in self.elements:
            e.print_info()
            
    def print_result(self):
        print('-----RESULT-----')
        
        global_max_observed_queue_length = 0
        global_mean_queue_length_accumulator = 0
        global_failure_probability_accumulator = 0
        global_max_load = 0
        global_mean_load_accumulator = 0
        num_of_processors = 0
        
        for e in self.elements:
            e.print_result()
            if isinstance(e, Process):
                num_of_processors += 1
                mean_queue_length = e.mean_queue_length / self.tcurr
                failure_probability = e.failure / (e.quantity + e.failure) if (e.quantity + e.failure) != 0 else 0
                mean_load = e.quantity / self.max_time
                
                global_mean_queue_length_accumulator += mean_queue_length
                global_failure_probability_accumulator += failure_probability
                global_mean_load_accumulator += mean_load
                
                if e.max_obs_queue_length > global_max_observed_queue_length:
                    global_max_observed_queue_length = e.max_obs_queue_length
                    
                if mean_load > global_max_load:
                    global_max_load = mean_load
                
                if self.print_logs:
                    print(f"Average queue length: {mean_queue_length}")
                    print(f"Failure probability: {failure_probability}")
                    print(f"Average load: {mean_load}")
                    print()
                
        global_mean_queue_length = global_mean_queue_length_accumulator / num_of_processors
        global_failure_probability = global_failure_probability_accumulator / num_of_processors
        global_mean_load = global_mean_load_accumulator / num_of_processors
        
        if self.print_logs:
            print(f"Global max observed queue length: {global_max_observed_queue_length}")
            print(f"Global mean queue length: {global_mean_queue_length}")
            print(f"Global failure probability: {global_failure_probability}")
            print(f"Global max load: {global_max_load}")
            print(f"Global mean load: {global_mean_load}")
            print()
        
        return {
            "global_max_observed_queue_length": global_max_observed_queue_length,
            "global_mean_queue_length": global_mean_queue_length,
            "global_failure_probability": global_failure_probability,
            "global_max_load": global_max_load,
            "global_mean_load": global_mean_load
        }

In [130]:
class SimModel():
    def __init__(self):
        c1 = Create(delay_mean=2.0, name='CREATOR_1', distribution='exp')
        p1 = Process(maxqueue=2, delay_mean=4.0, name='PROCESSOR1', distribution='exp')

        c1.next_elements = [p1]
        elements = [c1, p1]
        model = Model(elements, print_logs=True)
        model.simulate(10)

In [131]:
Element.id_curr = 0
s = SimModel()


It`s time for event in CREATOR_1; time=0.0
CREATOR_1: state=0; quantity=1; tnext=2.575648929427142
PROCESSOR1: state=1; quantity=0; tnext=1.6562314031379564
failure=0

It`s time for event in PROCESSOR1; time=1.6562314031379564
CREATOR_1: state=0; quantity=1; tnext=2.575648929427142
PROCESSOR1: state=0; quantity=1; tnext=inf
failure=0

It`s time for event in CREATOR_1; time=2.575648929427142
CREATOR_1: state=0; quantity=2; tnext=8.329135606976374
PROCESSOR1: state=1; quantity=1; tnext=5.951934382331398
failure=0

It`s time for event in PROCESSOR1; time=5.951934382331398
CREATOR_1: state=0; quantity=2; tnext=8.329135606976374
PROCESSOR1: state=0; quantity=2; tnext=inf
failure=0

It`s time for event in CREATOR_1; time=8.329135606976374
CREATOR_1: state=0; quantity=3; tnext=9.041218606349815
PROCESSOR1: state=1; quantity=2; tnext=10.892567650019643
failure=0

It`s time for event in CREATOR_1; time=9.041218606349815
CREATOR_1: state=0; quantity=4; tnext=9.092187111456033
PROCESSOR1: state=

In [132]:
class SimModel2():
    def __init__(self):
        c1 = Create(delay_mean=2.0, name='CREATOR1', distribution='exp')
        p1 = Process(maxqueue=5, delay_mean=1.0, name='PROCESSOR1', distribution='exp')
        p2 = Process(maxqueue=5, delay_mean=1.0, name='PROCESSOR2', distribution='exp')
        p3 = Process(maxqueue=5, delay_mean=1.0, name='PROCESSOR3', distribution='exp')
        p4 = Process(maxqueue=5, delay_mean=1.0, name='PROCESSOR4', distribution='exp')
        
        c1.next_elements = [p1]
        p1.next_elements = [p2, p3]
        p3.next_elements = [p4]
#         p4.next_elements = [p1]
        
        elements = [c1, p1, p2, p3, p4]

        model = Model(elements, print_logs=True)
        model.simulate(10)

In [133]:
Element.id_curr = 0
s = SimModel2()


It`s time for event in CREATOR1; time=0.0
CREATOR1: state=0; quantity=1; tnext=0.24217651188977138
PROCESSOR1: state=1; quantity=0; tnext=0.22505459412066414
failure=0
PROCESSOR2: state=0; quantity=0; tnext=inf
failure=0
PROCESSOR3: state=0; quantity=0; tnext=inf
failure=0
PROCESSOR4: state=0; quantity=0; tnext=inf
failure=0

It`s time for event in PROCESSOR1; time=0.22505459412066414
CREATOR1: state=0; quantity=1; tnext=0.24217651188977138
PROCESSOR1: state=0; quantity=1; tnext=inf
failure=0
PROCESSOR2: state=1; quantity=0; tnext=1.619132041894779
failure=0
PROCESSOR3: state=1; quantity=0; tnext=0.5521674813953692
failure=0
PROCESSOR4: state=0; quantity=0; tnext=inf
failure=0

It`s time for event in CREATOR1; time=0.24217651188977138
CREATOR1: state=0; quantity=2; tnext=3.1181769323578723
PROCESSOR1: state=1; quantity=1; tnext=1.36104849776744
failure=0
PROCESSOR2: state=1; quantity=0; tnext=1.619132041894779
failure=0
PROCESSOR3: state=1; quantity=0; tnext=0.5521674813953692
failure

**Побудуємо таблицю верифікації:**

In [134]:
from IPython.display import clear_output

In [135]:
n_param = 10

delay_create_list = [1]*n_param + [1]*n_param #list(range(1, n_param+1))
delay_process_list = [1]*n_param + [1]*n_param #list(range(1, n_param+1))
maxQ_list = [3]*n_param + [5]*n_param #list(range(1, n_param+1))

time_modeling_list = [i*100 for i in range(1, n_param+1)] + [i*100 for i in range(1, n_param+1)]

# np.random.shuffle(delay_create_list)
# np.random.shuffle(delay_process_list)
# np.random.shuffle(maxQ_list)
# np.random.shuffle(time_modeling_list)

df = pd.DataFrame()
rows = []

for i in range(n_param*2):
    Element.id_curr = 0
    c1 = Create(delay_mean=delay_create_list[i], name='CREATOR_1', distribution='exp')
    p1 = Process(maxqueue=maxQ_list[i], delay_mean=delay_process_list[i], name='PROCESSOR1', distribution='exp')

    c1.next_elements = [p1]
    elements = [c1, p1]
    model = Model(elements, print_logs=False)
    res = model.simulate(time_modeling_list[i])
    
    param = {'delay_mean_create': delay_create_list[i], 'delay_process_create': delay_process_list[i],
             'distribution': 'exp', 'maxqueue': maxQ_list[i], 'time_modeling': time_modeling_list[i]}
    
    rows.append({**param, **res})
df = df.append(rows)
clear_output()

**Проведемо верифікацію, використовуючи побудовану таблицю:**

In [136]:
df

,delay_mean_create,delay_process_create,distribution,maxqueue,time_modeling,global_max_observed_queue_length,global_mean_queue_length,global_failure_probability,global_max_load,global_mean_load
0,1,1,exp,3,100,3,0.000000,0.265306,0.720000,0.720000
1,1,1,exp,3,200,3,0.004056,0.145729,0.850000,0.850000
2,1,1,exp,3,300,3,0.000000,0.236196,0.830000,0.830000
3,1,1,exp,3,400,3,0.000640,0.170918,0.812500,0.812500
4,1,1,exp,3,500,3,0.000000,0.201566,0.816000,0.816000
5,1,1,exp,3,600,3,0.000000,0.194719,0.813333,0.813333
6,1,1,exp,3,700,3,0.001869,0.150000,0.825714,0.825714
7,1,1,exp,3,800,3,0.000000,0.184106,0.770000,0.770000
8,1,1,exp,3,900,3,0.001261,0.205184,0.817778,0.817778
9,1,1,exp,3,1000,3,0.002368,0.184995,0.793000,0.793000


## Conclusion

**В данній лабораторній роботі ми реалізували алгоритм імітації простої моделі обслуговування, використовуючи спосіб, який орієнтований на події (аби уникнути випадку, коли дві події приходять в один час).**
+ за допомогою статичних методів була зібрана інформація про роботу/поведінку моделі
+ зміна віхдних параметрів моделі призводить до зміни вихідних значень
+ при досить великій кількості вхідинх параметрів важко проводити етап верифікації моделі

P.S. нижче наведено декілька графіків, які ілюструють залежність вихідних параметрів від вхідних